# Preprocessing and clustering 4k PBMCs

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc


Read in the count matrix into an [AnnData](https://anndata.readthedocs.io/en/latest/anndata.AnnData.html) object, which holds many slots for annotations and different representations of the data. It also comes with its own HDF5-based file format: `.h5ad`.

In [ ]:
adata = sc.read_10x_mtx(
    'path/to/your/file',  # read .mtx files from the directory
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)                              # write a cache file for faster subsequent reading

In [ ]:
adata #check data structure

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white') #setting resolution for the graphs

:::{note}
See [anndata-tutorials/getting-started](https://anndata-tutorials.readthedocs.io/en/latest/getting-started.html) for a more comprehensive introduction to `AnnData`.
:::

In [ ]:
adata

## Preprocessing

Show those genes that yield the highest fraction of counts in each single cell, across all cells.

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20, ) #checking top 20 genes that are highly expressed in all cells counts

Basic filtering:

In [ ]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
adata

Let's assemble some information about mitochondrial genes, which are important for quality control.

Citing from "Simple Single Cell" workflows [(Lun, McCarthy & Marioni, 2017)](https://master.bioconductor.org/packages/release/workflows/html/simpleSingleCell.html#examining-gene-level-metrics):

> High proportions are indicative of poor-quality cells (Islam et al. 2014; Ilicic et al. 2016), possibly because of loss of cytoplasmic RNA from perforated cells. The reasoning is that mitochondria are larger than individual transcript molecules and less likely to escape through tears in the cell membrane.

With `pp.calculate_qc_metrics`, we can compute many metrics very efficiently.

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True) # pp.calculate_qc_metrics, is a function for qc_metrics calculation
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True) #ncount=number of expressed genes, total counts= toal expression of each cell(based on UMI), and pct_count_mt reflect the percentage of expression from mitochondria (setting for mt filtration)

A violin plot of some of the computed quality measures:

* the number of genes expressed in the count matrix
* the total counts per cell
* the percentage of counts in mitochondrial genes

Remove cells that have too many mitochondrial genes expressed or too many total counts:

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt') # checking the percent of expression from mitocondria versus total  (settig for filtration)
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')# checking the number of genes versus total genes 

Actually do the filtering by slicing the `AnnData` object.

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 3500, :] # setting range from 2500-4500 depending on the distribution of scatters 
adata = adata[adata.obs.pct_counts_mt < 5, :] #common setting for mt filtration

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')
adata

Total-count normalize (library-size correct) the data matrix $\mathbf{X}$ to 10,000 reads per cell, so that counts become comparable among cells.

In [ ]:
## normalization expressions levels for comparison across different cells 
sc.pp.normalize_total(adata, target_sum=1e4) 
sc.pp.log1p(adata)

Logarithmize the data:

In [ ]:
adata.raw = adata #unifying object of Anna for further analysis and visualization

Identify highly-variable genes.

In [ ]:
#Identify highly-variable genes.
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5) 
sc.pl.highly_variable_genes(adata) #ploting highly variable genes

In [ ]:
#listing top expressed genes
highly_variable_genes = adata.var[adata.var['highly_variable']].index
print(highly_variable_genes[:10]) 

In [ ]:
#retains highly variable genes for further analysis
adata = adata[:, adata.var.highly_variable]
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10) #sclae gene to unit variance
adata # checking filteration result

## Principal component analysis

Reduce the dimensionality of the data by running principal component analysis (PCA), which reveals the main axes of variation and denoises the data.

In [ ]:
# Principal component analysis
sc.tl.pca(adata, svd_solver='arpack')
#sc.pl.pca(adata, color='CST3')

We can make a scatter plot in the PCA coordinates, but we will not use that later on.

Let us inspect the contribution of single PCs to the total variance in the data. This gives us information about how many PCs we should consider in order to compute the neighborhood relations of cells, e.g. used in the clustering function  `sc.tl.louvain()` or tSNE `sc.tl.tsne()`. In our experience, often a rough estimate of the number of PCs does fine.

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True) # checking the covariance of each PC for total 

Save the result.

In [ ]:
adata.write(results_file) #store file 

In [ ]:
adata

## Computing the neighborhood graph

Let us compute the neighborhood graph of cells using the PCA representation of the data matrix. You might simply use default values here. For the sake of reproducing Seurat's results, let's take the following values.

In [ ]:
#neighborhood graph calculation and exmaine via biomarkers
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40) # parameters varied by datasets, but here is taking the default values.
sc.tl.umap(adata)
sc.pl.umap(adata, color=['CST3', 'NKG7', 'PPBP'])

## Embedding the neighborhood graph

We suggest embedding the graph in two dimensions using UMAP ([McInnes et al., 2018](https://arxiv.org/abs/1802.03426)), see below. It is potentially more faithful to the global connectivity of the manifold than tSNE, i.e., it better preserves trajectories. In some ocassions, you might still observe disconnected clusters and similar connectivity violations. They can usually be remedied by running:

```Python
sc.tl.paga(adata)
sc.pl.paga(adata, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(adata, init_pos='paga')
```

As we set the `.raw` attribute of `adata`, the previous plots showed the "raw" (normalized, logarithmized, but uncorrected) gene expression. You can also plot the scaled and corrected gene expression by explicitly stating that you don't want to use `.raw`.

In [ ]:
sc.pl.umap(adata, color=['CST3', 'NKG7', 'PPBP'], use_raw=False) #using normalize, logarithmized, and corrected datasets by setting raw as False, 

## Clustering the neighborhood graph

As with Seurat and many other frameworks, we recommend the Leiden graph-clustering method (community detection based on optimizing modularity) by [Traag *et al.* (2018)](https://scanpy.readthedocs.io/en/latest/references.html#traag18). Note that Leiden clustering directly clusters the neighborhood graph of cells, which we already computed in the previous section.

In [ ]:
#using Leiden graph-based clustering method
sc.tl.leiden(adata) #calculate 
sc.pl.umap(adata, color=['leiden']) #plot

Plot the clusters, which agree quite well with the result of Seurat.

Save the result.

In [ ]:
adata.write(results_file)

## Finding marker genes

Let us compute a ranking for the highly differential genes in each cluster. For this, by default, the `.raw` attribute of AnnData is used in case it has been initialized before. The simplest and fastest method to do so is the t-test.

In [ ]:
#ranking top_25 genes via t-test for efficiency
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
sc.settings.verbosity = 3  # reduce the verbosity

In [ ]:
##Finding Marker genes
#checking for clusters and p-values
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']}).head(10)

The result of a [Wilcoxon rank-sum (Mann-Whitney-U)](https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test) test is very similar. We recommend using the latter in publications, see e.g., [Sonison & Robinson (2018)](https://doi.org/10.1038/nmeth.4612). You might also consider much more powerful differential testing packages like MAST, limma, DESeq2 and, for python, the recent diffxpy.

Save the result.

In [ ]:
adata.write(results_file)
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']}).head(10)

As an alternative, let us rank genes using logistic regression. For instance, this has been suggested by [Natranos et al. (2018)](https://doi.org/10.1101/258566). The essential difference is that here, we use a multi-variate appraoch whereas conventional differential tests are uni-variate. [Clark et al. (2014)](https://doi.org/10.1186/1471-2105-15-79) has more details.

In [ ]:
marker_genes = ['IL7R', 'CD79A', 'MS4A1', 'CD8A', 'CD8B', 'LYZ', 'CD14',
                'LGALS3', 'S100A8', 'GNLY', 'NKG7', 'KLRB1',
                'FCGR3A', 'MS4A7', 'FCER1A', 'CST3', 'PPBP']
adata = sc.read(results_file)
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(5)

In [ ]:
#examine subclusters by leiden
for i in adata.obs['leiden'].cat.categories:
  number = len(adata.obs[adata.obs['leiden']==i])
  print('the number of category {} is {}'.format(i,number))

In [ ]:
#remove genes with minimal cell counts
adata = adata[adata.obs[adata.obs['leiden'].astype(int)<11].index]
adata

In [ ]:
# examine clustering result after the removal
for i in adata.obs['leiden'].cat.categories:
  number = len(adata.obs[adata.obs['leiden']==i])
  print('the number of category {} is {}'.format(i,number))
sc.pl.umap(adata, color=['leiden'], wspace=0.4, show=False)

In [ ]:
adata #checking status

In [ ]:
#Annotation:
marker_genes = ['CD3D', 'CD3E', 'CD3G','NKG7', 'KLRB1', 'MS4A1', 'CD79A', 'CD79B', 'CD68', 'CD1C']
marker_genes1 = ['IL7R', 'CD79A', 'MS4A1', 'CD8A', 'CD8B', 'LYZ', 'CD14',
                'LGALS3', 'S100A8', 'GNLY', 'NKG7', 'KLRB1',
                'FCGR3A', 'MS4A7', 'FCER1A', 'CST3', 'PPBP']

With the exceptions of *IL7R*, which is only found by the t-test and *FCER1A*, which is only found by the other two appraoches, all marker genes are recovered in all approaches.

Louvain Group | Markers | Cell Type
---|---|---
0 | IL7R | CD4 T cells
1 | CD14, LYZ | CD14+ Monocytes
2 | MS4A1 |	B cells
3 | CD8A |	CD8 T cells
4 | GNLY, NKG7 | 	NK cells
5 | FCGR3A, MS4A7 |	FCGR3A+ Monocytes
6 | FCER1A, CST3 |	Dendritic Cells
7 | PPBP |	Megakaryocytes

Let us also define a list of marker genes for later reference.

In [ ]:
sc.pl.dotplot(adata, marker_genes, groupby='leiden');
sc.pl.dotplot(adata, marker_genes1, groupby='leiden');

In [ ]:
cluster2annotation = {
    '0': 'CD4 T',
    '1': 'CD14 Monocytes',
    '2': 'CD8T',
    '3': 'CD4T',
    '4': 'B cells',
    '5': 'NK ',
    '6': 'NK cells',
    '7': 'B cells',
    '8': 'NK cells',
    '9': 'Dendritic cells',
    '10': 'FCGR3A+ Monocytes'
}
adata.obs['major_celltype'] = adata.obs['leiden'].map(cluster2annotation).astype('category')
#new_cluster_names = ['CD4 T', 'CD 14 Monocytes', 'CD8T', 'CD4T','B cell','CD8T', 'NK cell', 'B cell', 'NK cell', 'Dendritic', 'FCGR3A Monocytes']
#adata.rename_categories('leiden', new_cluster_names)

Reload the object that has been save with the Wilcoxon Rank-Sum test result.

In [ ]:
#adata = sc.read(results_file)

Show the 10 top ranked genes per cluster 0, 1, ..., 7 in a dataframe.

In [ ]:
sc.tl.dendrogram(adata,groupby='major_celltype')
sc.pl.dotplot(
    adata,
    marker_genes_dict,
    groupby='major_celltype',
    dendrogram=True,
    color_map="Blues",
    swap_axes=False,
    use_raw=True,
    standard_scale="var",
)

Get a table with the scores and groups.

In [ ]:
ax=sc.pl.embedding(
    adata,
    basis="X_umap",
    color='major_celltype',
    title='RNA-seq',
    frameon=False,
    ncols=3,
    #save='_figure1_celltype.png',
    return_fig=True,
    show=False,
)

Compare to a single cluster: 

In [ ]:
sc.pl.umap(adata, color='major_celltype', legend_loc='on data', title='PMC4K_RNA-seq', frameon=False, save='.pdf')

If we want a more detailed view for a certain group, use `sc.pl.rank_genes_groups_violin`.

In [ ]:
sc.pl.stacked_violin(adata, marker_genes, groupby='leiden', rotation=90);

Reload the object with the computed differential expression (i.e. DE via a comparison with the rest of the groups):

In [ ]:
results_file = 'write/pbmc4k_Raw_2nd.h5ad'  # naming file for analysis results storage

In [ ]:
adata.write(results_file, compression='gzip')  # `compression='gzip'` saves disk space, but slows down writing and subsequent reading

In [ ]:
adata.raw.to_adata().write('./write/pbmc4k_2nd_run.h5ad')

If you want to compare a certain gene across groups, use the following.

In [ ]:
# Export single fields of the annotation of observations
adata.obs[['n_counts', 'louvain_groups']].to_csv('./write/pbmc4k_corrected_louvain_groups.csv')

# Export single columns of the multidimensional annotation
#adata.obsm.to_df()[['X_pca1', 'X_pca2']].to_csv('./write/pbmc4k_corrected_X_pca.csv')

# Or export everything except the data using `.write_csv`.
# Set `skip_data=False` if you also want to export the data.
# adata.write_csvs(results_file[:-5], )